# Pandas 데이터 병합
---

In [63]:
# 필요 라이브러리 import
import pandas as pd
import numpy as np
import seaborn as sns
import random


In [64]:
# 변수 실행시 마지막 변수만 보여주던 것을 모든 변수를 보여주도록 설정 변경

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"


### DataFrame 병합
* 하나의 데이터만 가지고 분석하기도 하지만, 여러 데이터를 합쳐서 하나의 데이터로 만든 후에 분석하기도 한다.

* 데이터 결합(join)
    * 두 개 이상의 데이터 세트를 결합하여 한 개의 데이터 세트로 만드는 과정
    
* 데이터 결합 방식
    * 단순 결합 : `concat` 사용
    * 공통 필드 결합 : [merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)
        * `left_join()` / `inner_join()` / `full_join()` / `outer_join()`
        
* pandas는 두 개 이상의 데이터프레임을 하나로 합치는 병합(merge)이나 연결(concat)을 지원한다.


#### `merge`를 활용한 데이터 프레임 병합


* 형식 
    * `dataframe.merge(df1, how=병합방식)` : 두 dataframe을 병합시켜 준다.
    * 기본은 inner join : 양쪽에 동일하게 존재하는 키만 표시한다.
    * key : 기준열을 의미
        * 실제 데이터 필드이거나 행 인덱스일 수 있다.
    * 병합방식
        * inner : 양쪽 dataframe에서 모두 키가 존재하는 data만 표시
        * outer : 양쪽 모두를 포함 (한쪽에만 있는 경우 옆은 NULL 처리)
        * left, right : 왼쪽 / 오른쪽 key를 기준으로 병합

In [65]:
#예시 df 생성 - 고객 정보를 담고 있는 df
df1 =pd.DataFrame({
    '고객번호' : [1001,1002,1003,1004,1005,1006,1007],
    '이름' : ['둘리','도우너','또치','길동','희동','마이콜','영희']
        },
    columns=['고객번호','이름'])
df1

#예제 df 생성 - 예금 정보 df
df2 = pd.DataFrame({
    '고객번호':[1001,1001,1005,1006,1008,1001],
    '금액' : [10000,20000,15000,5000,100000,30000]
},columns=['고객번호','금액'])
df2


,고객번호,이름
0,1001,둘리
1,1002,도우너
2,1003,또치
3,1004,길동
4,1005,희동
5,1006,마이콜
6,1007,영희


,고객번호,금액
0,1001,10000
1,1001,20000
2,1005,15000
3,1006,5000
4,1008,100000
5,1001,30000


##### inner_join

* merge 명령으로 두 df를 병합하는 문법
* 모든 인수 생략 가능(병합할 df 제외) - 공통 이름('고객번호')을 갖고 있는 열이 존재해야 함
* '고객번호' 공통이름 필드가 있음 - key로 설정
* 양쪽에 모두 존재하는 키의 data만 보여줌 (inner join)

In [66]:

## 공통 필드 존재 확인 
df1.head(1)
df2.head(1)

## 방법 1)기준이 되는 df 객체의 메서드를 사용함
df1.merge(df2)

## 방법 2) pd.merge(기준df, df2)
pd.merge(df1, df2)

,고객번호,이름
0,1001,둘리


,고객번호,금액
0,1001,10000


,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000


##### outer_join
* 키 값이 한쪽에만 있어도 데이터를 출력 (한쪽에만 있는 경우에는 Null 처리)
    * 어느 한 df에만 데이터가 존재하고 다른 df에만 존재하지 않으면 해당 필드에 NaN 표시
* `how='outer'` 옵션으로 가능


In [67]:
pd.merge(df1, df2, how='outer')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN
9,1008,NaN,100000.0


##### left / right join
* left : 왼쪽 df에만 있는 모든 키의 데이터 표시
* right : 오른쪽 df에믄 있는 모든 키의 데이터 표시


In [68]:
pd.merge(df1, df2, how='left')
pd.merge(df1, df2, how='right')

,고객번호,이름,금액
0,1001,둘리,10000.0
1,1001,둘리,20000.0
2,1001,둘리,30000.0
3,1002,도우너,NaN
4,1003,또치,NaN
5,1004,길동,NaN
6,1005,희동,15000.0
7,1006,마이콜,5000.0
8,1007,영희,NaN


,고객번호,이름,금액
0,1001,둘리,10000
1,1001,둘리,20000
2,1001,둘리,30000
3,1005,희동,15000
4,1006,마이콜,5000
5,1008,NaN,100000


* df에 key 값이 같은 데이터가 여러 개 있는 경우에는, 모든 경우의 수를 따져서 조합을 만들어낸다.


In [69]:

#예제 df 생성 
#열: 품종, 꽃잎길이
df1 = pd.DataFrame({
    '품종':['setosa','setosa','virginica','virginica'],
    '꽃잎길이':[1.4,1.3,1.5,1.3]
}, columns=['품종','꽃잎길이'])
df1


#열 : 품종, 꽃잎너비
df2 = pd.DataFrame({
    '품종': ['setosa','virginica','virginica','ersicolor'],
    '꽃잎너비':[0.4,0.3,0.5,0.3]
},columns=['품종','꽃잎너비'])
df2


,품종,꽃잎길이
0,setosa,1.4
1,setosa,1.3
2,virginica,1.5
3,virginica,1.3


,품종,꽃잎너비
0,setosa,0.4
1,virginica,0.3
2,virginica,0.5
3,ersicolor,0.3


df1과 df2를 병합한다.  
위의 데이터에서 키 값 setosa에 대해 df1은 1.4와 1.3 2개의 데이터가 있고, df2에는 0.4라는 1개의 데이터가 있으므로,  
병합 데이터에는 setosa가 (1.4, 0.4) (1.3, 0.4)의 2 경우가 표현된다.  
  
키 값 virginica의 경우에는 df1에 2개 df2 2개의 데이터가 있으므로,   
2개와 2개의 조합에 의해 4개의 데이터가 표현된다.

In [70]:
pd.merge(df1, df2)

,품종,꽃잎길이,꽃잎너비
0,setosa,1.4,0.4
1,setosa,1.3,0.4
2,virginica,1.5,0.3
3,virginica,1.5,0.5
4,virginica,1.3,0.3
5,virginica,1.3,0.5


* key
    * 두 데이터 프레임에서 이름이 같은 열은 모두 키가 될 수 있다.
    * 따라서, 두 데이터프레임에 이름이 같은 열이 여러 개 존재하는 경우 병합할 key 이름을 명시해줘야한다.
    * 기준열을 직접 지정하는 방법은 `on=기준열 이름`으로 지정한다.
        * 반환 결과에 동일 필드명이 있을 경우에는 필드명_x, 필드명_y로 필드명을 변경해서 표현한다.

In [71]:
# 예제 df
df1 = pd.DataFrame({
    '고객명':['춘향','춘향','몽룡'],
    '날짜' : ['2018-01-01','2018-01-02','2018-01-01'],
    '데이터':[20000,30000,100000]
})
df1


df2 = pd.DataFrame({
    '고객명':['춘향','몽룡'],
    '데이터':['여자','남자']
})
df2


pd.merge(df1, df2, on='고객명')

,고객명,날짜,데이터
0,춘향,2018-01-01,20000
1,춘향,2018-01-02,30000
2,몽룡,2018-01-01,100000


,고객명,데이터
0,춘향,여자
1,몽룡,남자


,고객명,날짜,데이터_x,데이터_y
0,춘향,2018-01-01,20000,여자
1,춘향,2018-01-02,30000,여자
2,몽룡,2018-01-01,100000,남자


##### 공통 필드가 없는 경우
* 키가 되는 기준열이 두 데이터프레임에서 다르게 나타나면, `left_on`, `right_on` 인수를 사용해서 기준열을 명시해야 한다.

In [72]:
df1 = pd.DataFrame({
    '이름' :['영희','철수','철수'],
    '성적' :[90,80,80]
})
df2 = pd.DataFrame({
    '성명' :['영희','영희','철수'],
    '성적2':[100,80,90]
})
df1.head(1)
df2.head(1)


,이름,성적
0,영희,90


,성명,성적2
0,영희,100


In [73]:
# 양쪽 df의 기준이되는 열의 이름이 다름.

pd.merge(df1, df2, left_on='이름', right_on='성명')

# 양쪽의 필드명이 다르기 때문에, 출력 결과는 기준열로 설정한 두 필드 모두를 반환한다.


,이름,성적,성명,성적2
0,영희,90,영희,100
1,영희,90,영희,80
2,철수,80,철수,90
3,철수,80,철수,90


##### 일반 데이터열이 아닌 인덱스를 기준으로 merge하기
* 인덱스를 기준열로 사용할려면 `left_index=True`, `right_index=True`를 사용

In [74]:
df1 = pd.DataFrame({
    '도시': ['서울','서울','서울','부산','부산'],
    '연도': [2000,2005,2010,2000,2005],
    '인구':[9853972,9762546,9631482,3655437,3512547]    
})
df2=pd.DataFrame(
    np.arange(12).reshape((6,2)),
    index=[['부산','부산','서울','서울','서울','서울'],
          [2000,2005,2000,2005,2010,2015]],
    columns=['데이터1','데이터2']
)
df1
df2


,도시,연도,인구
0,서울,2000,9853972
1,서울,2005,9762546
2,서울,2010,9631482
3,부산,2000,3655437
4,부산,2005,3512547


데이터1  데이터2
부산 2000     0     1
   2005     2     3
서울 2000     4     5
   2005     6     7
   2010     8     9
   2015    10    11

In [75]:
# 기준열로 삼은 index는 실제 data가 아니기 때문에 표현되지 않는다.

pd.merge(df1, df2, left_on=['도시', '연도'], right_index=True)

,도시,연도,인구,데이터1,데이터2
0,서울,2000,9853972,4,5
1,서울,2005,9762546,6,7
2,서울,2010,9631482,8,9
3,부산,2000,3655437,0,1
4,부산,2005,3512547,2,3


* 일반 데이터에는 기준열이 없고, merge할 두 df 모두 index를 기준열로 설정해야 하는 경우

In [76]:

df1 = pd.DataFrame(
[[1.,2.],[3.,4.],[5.,6.]],
index=['a','c','e'],
columns=['서울','부산'])
df1

df2=pd.DataFrame(
[[7.,8.],[9.,10.],[11.,12.],[13.,14.]],
    index=['b','c','d','e'],
columns=['대구','광주'])
df2


# 위에서 작성한 두 df를 인덱스를 기준열로 설정하고 병합
# 양쪽 데이터프레임에서 key가 모두 인덱스인 경우

pd.merge(df1, df2, how='outer', left_index=True, right_index=True)

,서울,부산
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,대구,광주
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


* `merge` 대신에 `join` 메서드를 사용할 수 있다.

In [77]:
df1.join(df2, how='outer')

,서울,부산,대구,광주
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### 데이터 병합 연습문제
* 두 개의 데이터프레임을 만들고, `merge`를 사용해 병합
* 단, 데이터프레임은 다음 조건을 만족해야 한다.
    1. 각각 5*5 이상의 크기를 가진다.
    2. 공통 열을 하나 이상 갖는다.
    3. `merge`의 경우를 inner, outer, left, right 4개의 형태로 출력할 것.
    4. 지정된 인덱스와 컬럼명을 갖는다.

In [78]:
data = [[22, 60.1, 170.5, '남', '서울'], 
       [45, 51.3, 157.3, '여', '부산'],
       [22, 68.3, 180.1, '남', '대구'],
       [33, 88.3, 190.2, '남', '제주'],
       [27, 48.3, 160.1, '여', '강릉']]

df1 = pd.DataFrame(data, 
                  index=['홍길동', '이몽룡', '성춘향', '변학도', '김연아'],
                  columns=['나이', '몸무게', '키', '성별', '주소'])

df1

data = [[22, 60.1, 170.5, '남', '서울'], 
       [45, 51.3, 157.3, '여', '부산'],
       [22, 68.3, 180.1, '남', '울산'],
       [33, 88.3, 190.2, '남', '제주'],
       [27, 48.3, 160.1, '여', '광주']]

df2 = pd.DataFrame(data, 
                  index=[1, 2, 3, 4, 5],
                  columns=['나이', '몸무게', '키', '성별', '지역'])

df2

,나이,몸무게,키,성별,주소
홍길동,22,60.1,170.5,남,서울
이몽룡,45,51.3,157.3,여,부산
성춘향,22,68.3,180.1,남,대구
변학도,33,88.3,190.2,남,제주
김연아,27,48.3,160.1,여,강릉


,나이,몸무게,키,성별,지역
1,22,60.1,170.5,남,서울
2,45,51.3,157.3,여,부산
3,22,68.3,180.1,남,울산
4,33,88.3,190.2,남,제주
5,27,48.3,160.1,여,광주


In [79]:
# inner_join
pd.merge(df1, df2, left_on='주소', right_on='지역')

# outer_join
pd.merge(df1, df2, left_on='주소', right_on='지역', how='outer')

# left_join
pd.merge(df1, df2, left_on='주소', right_on='지역', how='left')

# right_join
pd.merge(df1, df2, left_on='주소', right_on='지역', how='right')

,나이_x,몸무게_x,키_x,성별_x,주소,나이_y,몸무게_y,키_y,성별_y,지역
0,22,60.1,170.5,남,서울,22,60.1,170.5,남,서울
1,45,51.3,157.3,여,부산,45,51.3,157.3,여,부산
2,33,88.3,190.2,남,제주,33,88.3,190.2,남,제주


,나이_x,몸무게_x,키_x,성별_x,주소,나이_y,몸무게_y,키_y,성별_y,지역
0,22.0,60.1,170.5,남,서울,22.0,60.1,170.5,남,서울
1,45.0,51.3,157.3,여,부산,45.0,51.3,157.3,여,부산
2,22.0,68.3,180.1,남,대구,NaN,NaN,NaN,NaN,NaN
3,33.0,88.3,190.2,남,제주,33.0,88.3,190.2,남,제주
4,27.0,48.3,160.1,여,강릉,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,22.0,68.3,180.1,남,울산
6,NaN,NaN,NaN,NaN,NaN,27.0,48.3,160.1,여,광주


,나이_x,몸무게_x,키_x,성별_x,주소,나이_y,몸무게_y,키_y,성별_y,지역
0,22,60.1,170.5,남,서울,22.0,60.1,170.5,남,서울
1,45,51.3,157.3,여,부산,45.0,51.3,157.3,여,부산
2,22,68.3,180.1,남,대구,NaN,NaN,NaN,NaN,NaN
3,33,88.3,190.2,남,제주,33.0,88.3,190.2,남,제주
4,27,48.3,160.1,여,강릉,NaN,NaN,NaN,NaN,NaN


,나이_x,몸무게_x,키_x,성별_x,주소,나이_y,몸무게_y,키_y,성별_y,지역
0,22.0,60.1,170.5,남,서울,22,60.1,170.5,남,서울
1,45.0,51.3,157.3,여,부산,45,51.3,157.3,여,부산
2,NaN,NaN,NaN,NaN,NaN,22,68.3,180.1,남,울산
3,33.0,88.3,190.2,남,제주,33,88.3,190.2,남,제주
4,NaN,NaN,NaN,NaN,NaN,27,48.3,160.1,여,광주


#### `concat()` 을 사용한 데이터 연결
* [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)
* `concat()`을 사용하면 기준열 없이 데이터를 연결한다.
* default는 위 아래로 데이터 행을 연결, `axis` 속성을 변경하면 열을 연결한다.
* 단순히 두 시리즈나 데이터프레임을 연결하기 때문에, 인덱스 값이 중복될 수 있다.

In [80]:
# 두 시리즈 데이터 연결
s1 = pd.Series([0, 1], index=['A', 'B'])
s2 = pd.Series([2, 3, 4], index=['A', 'B', 'C'])

s1
s2

# 두 시리즈 연결 - 아래 행에 연결
pd.concat([s1, s2])

# 열에 연결 - 열에 연결
pd.concat([s1, s2], axis=1)

A    0
B    1
dtype: int64

A    2
B    3
C    4
dtype: int64

A    0
B    1
A    2
B    3
C    4
dtype: int64

,0,1
A,0.0,2
B,1.0,3
C,NaN,4


* `concat()`시 인덱스가 각자 다 다른 경우
    * `ignore_index` 옵션을 통해서도 인덱스가 모두 같은 경우에도 같은 형태로 반환 가능


In [81]:
# 데이터 프레임에 대한 concat() 연결

df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'], 
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])
df1
df2
df3


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [82]:
# 1) 위 아래로 단순 병합 (행 기준)

result = pd.concat([df1, df2, df3], axis=0)
result

# 2) 양 옆으로 병합 (열 기준)
result = pd.concat([df1, df2, df3], axis=1)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


,A,B,C,D,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,A4,B4,C4,D4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,A5,B5,C5,D5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,A6,B6,C6,D6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,A7,B7,C7,D7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A8,B8,C8,D8
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A9,B9,C9,D9


* `concat()` 시 인덱스가 모두 같은 경우

In [83]:
# 데이터 프레임에 대한 concat() 연결

df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'], 
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[0, 1, 2, 3])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[0, 1, 2, 3])
df1
df2
df3


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


,A,B,C,D
0,A8,B8,C8,D8
1,A9,B9,C9,D9
2,A10,B10,C10,D10
3,A11,B11,C11,D11


In [84]:
# 1) 위 아래로 단순 병합 (행 기준)

result = pd.concat([df1, df2, df3], axis=0)
result



,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7
0,A8,B8,C8,D8
1,A9,B9,C9,D9


* 인덱스가 중복되어서 나온다.
    * pandas는 인덱스가 중복되면, 행/열을 구분하기 위해서 다중 인덱스를 설정한다.
    * `keys` 매개변수로 상위 계층 인덱스를 생성 가능함

In [85]:
result = pd.concat([df1, df2, df3], keys=['x', 'y', 'z'])
result

A    B    C    D
x 0   A0   B0   C0   D0
  1   A1   B1   C1   D1
  2   A2   B2   C2   D2
  3   A3   B3   C3   D3
y 0   A4   B4   C4   D4
  1   A5   B5   C5   D5
  2   A6   B6   C6   D6
  3   A7   B7   C7   D7
z 0   A8   B8   C8   D8
  1   A9   B9   C9   D9
  2  A10  B10  C10  D10
  3  A11  B11  C11  D11

In [86]:
result.index
result.index.get_level_values(0)

MultiIndex([('x', 0),
            ('x', 1),
            ('x', 2),
            ('x', 3),
            ('y', 0),
            ('y', 1),
            ('y', 2),
            ('y', 3),
            ('z', 0),
            ('z', 1),
            ('z', 2),
            ('z', 3)],
           )

Index(['x', 'x', 'x', 'x', 'y', 'y', 'y', 'y', 'z', 'z', 'z', 'z'], dtype='object')

In [87]:
# 2) 양 옆으로 병합 (열 기준)
result = pd.concat([df1, df2, df3], axis=1)
result

,A,B,C,D,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4,A8,B8,C8,D8
1,A1,B1,C1,D1,A5,B5,C5,D5,A9,B9,C9,D9
2,A2,B2,C2,D2,A6,B6,C6,D6,A10,B10,C10,D10
3,A3,B3,C3,D3,A7,B7,C7,D7,A11,B11,C11,D11


* index를 기준으로 합치고, 없는 경우에는 NaN 처리(outer)


In [88]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'], 
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])

df4


,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [89]:
result = pd.concat([df1, df4], axis=1)
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


* 만약 둘 다 있는 행만 사용하고 싶으면, `join='inner'`옵션을 사용

In [90]:
result = pd.concat([df1, df4], axis=1, join='inner')
result

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


* `reindex()` : 기준 인덱스 지정 함수
    * `reindex(df1.index)` : df1의 index를 기준으로 병합(결합)

In [91]:
# left join과 비슷한 모양새를 보임
result = pd.concat([df1, df4], axis=1).reindex(df1.index)
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


* 두 df를 위 아래로 결합 시에 기존 index를 무시하고 새로 index를 부여하는 매개변수 : `ignore_index=True`

In [92]:
result = pd.concat([df1, df2, df3], ignore_index=True)
result
result.index

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


RangeIndex(start=0, stop=12, step=1)

#### 연습문제
* 어느 회사의 전반기 (1월 ~ 6월) 실적을 나타내는 데이터프레임과 후반기(7월 ~ 12월) 실적을 나타내는 데이터프레임을 작성한 후 합친다.(단순병합)
* 실적 정보는 '매출', '비용', '이익'으로 이루어진다.
* 이익 = 매출 - 비용 : 가공 필드 생성
* 또한, 1년 간의 총 실적을 마지막 열에 만든다.

In [130]:
np.random.seed(1)

# 1) 전반기(first_half), 후반기(second_half) 데이터프레임 생성

first_half = pd.DataFrame(np.random.randint(1, 100, size=(2, 6)),
                         index=['매출', '비용'],
                         columns=[str(i)+'월' for i in range(1, 7)])

first_half

second_half = pd.DataFrame(np.random.randint(1, 100, size=(2, 6)),
                          index=['매출', '비용'],
                          columns=[str(i)+'월' for i in range(7, 13)])
second_half

,1월,2월,3월,4월,5월,6월
매출,38,13,73,10,76,6
비용,80,65,17,2,77,72


,7월,8월,9월,10월,11월,12월
매출,7,26,51,21,19,85
비용,12,29,30,15,51,69


In [131]:
# 2) 두 데이터 병합
year_df = pd.concat([first_half, second_half], axis=1 )
year_df

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
매출,38,13,73,10,76,6,7,26,51,21,19,85
비용,80,65,17,2,77,72,12,29,30,15,51,69


In [132]:
# 3) 이익 필드 생성

year_df.loc['이익'] = year_df.loc['매출'] - year_df.loc['비용']
year_df

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
매출,38,13,73,10,76,6,7,26,51,21,19,85
비용,80,65,17,2,77,72,12,29,30,15,51,69
이익,-42,-52,56,8,-1,-66,-5,-3,21,6,-32,16


In [133]:
# 4) 실적 필드 생성

year_df['결산'] =  year_df.sum(1)
year_df

,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,결산
매출,38,13,73,10,76,6,7,26,51,21,19,85,425
비용,80,65,17,2,77,72,12,29,30,15,51,69,519
이익,-42,-52,56,8,-1,-66,-5,-3,21,6,-32,16,-94
